# Part 4: Using GPU acceleration with PyTorch

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try:
    import torchbearer
except:
    !pip install torchbearer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [2]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 2.5445, -0.0966,  0.5024,  ...,  0.4138,  2.7817, -0.3940],
        [ 0.1413,  1.4629, -1.4513,  ...,  1.2770,  1.1097,  2.4314],
        [-0.0966, -0.1375, -0.6239,  ...,  0.4162,  0.7333,  0.2202],
        ...,
        [-0.6581, -0.1115,  0.8592,  ...,  0.5843, -0.5972, -1.3264],
        [ 0.0366,  0.7289,  0.2949,  ..., -0.4553, -0.1566, -0.5714],
        [-0.1473, -0.3592, -0.0854,  ..., -0.3425,  3.5066, -0.0156]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [1]:
import torch

print("PyTorch 版本:", torch.__version__)
print("CUDA 是否可用:", torch.cuda.is_available())
print("CUDA 版本:", torch.version.cuda)
print("GPU 数量:", torch.cuda.device_count())
print("当前 GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

PyTorch 版本: 2.5.1+cu124
CUDA 是否可用: True
CUDA 版本: 12.4
GPU 数量: 1
当前 GPU: Tesla T4
tensor([[ 7.5835e-01, -8.7935e-01,  8.3842e-01,  ...,  3.5861e-01,
          2.2421e+00,  1.0415e+00],
        [ 3.6426e-02, -6.6319e-01, -3.3687e-01,  ...,  3.2979e+00,
         -3.5860e-01,  5.2358e-02],
        [-3.7338e-01,  1.9043e+00, -2.7184e+00,  ..., -1.1540e+00,
         -1.3089e+00,  4.4602e-01],
        ...,
        [ 5.4458e-01,  1.7210e-01,  1.1617e-01,  ...,  3.5510e-01,
         -2.0662e+00, -1.4916e-02],
        [ 5.0272e-01,  7.3801e-01,  6.7912e-02,  ..., -2.1648e+00,
          3.9979e-04,  3.6553e-02],
        [-2.0230e-01,  6.5338e-02,  3.7142e-01,  ...,  1.0823e-01,
         -4.6158e-01,  3.0549e-01]], device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[ 1.4801, -0.4365, -0.7820,  ...,  0.2495, -0.0638, -0.7991],
        [-1.5582,  0.2488, -2.1913,  ..., -0.0412,  0.0960, -0.5931],
        [-0.2589, -0.1030, -0.4060,  ...,  0.4875, -0.8480,  1.4511],
        ...,
        [ 0.1856, -0.0542, -0.0054,  ...,  0.1664, -0.1849, -0.0546],
        [-0.7798, -1.5766, -1.0208,  ...,  0.8521, -0.2228, -0.0852],
        [-0.0213, -0.7580, -1.2363,  ..., -0.0506, -0.2185, -0.0596]],
       device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

GPUs have thousands of cores designed for parallel processing, making them highly efficient for tasks like matrix multiplication and deep learning.GPUs are specialized for performing floating-point arithmetic, which is essential for machine learning and scientific computations.

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

 High-performance GPUs consume a lot of power, making large-scale training expensive.